In [419]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from asset_data_module import read_close_prices_all_merged
from features import make_feature_windows
tf.keras.utils.set_random_seed(42)


In [435]:
# markets = ['dow30']
markets = ['commodities']
start_date, end_date = "2022-01-01", "2025-11-28"
# start_date, end_date = "2024-06-01", "2025-11-28"

_, close_df = read_close_prices_all_merged(markets, after_date=start_date)
close_df = close_df.loc[:end_date]

rolling = make_feature_windows(
    close_prices=close_df,
    lookback=60,
    horizon=1,
    days_per_week=2
)
close_df.shape, len(rolling)

((984, 3), 432)

In [436]:
print(rolling[0]['past_weekly_returns'].shape)
rolling[0]['past_weekly_returns'].head()

(60, 3)


,commodities:GC=F,commodities:HG=F,commodities:SI=F
0,0.013907,-0.002268,0.015673
1,-0.015242,-0.000341,-0.033202
2,0.011948,0.003967,0.018231
3,0.001429,0.025352,0.015187
4,-0.004899,-0.035469,0.014236


In [437]:
rolling[0]['X_feat'].head()

,mom_1w,mom_4w,mom_12w,vol_1w,vol_4w,sharpe_1w,sharpe_4w,vol_ratio,max_drawdown
commodities:GC=F,0.762712,0.614606,0.783788,-0.898660,-1.145738,-0.080740,0.610305,-0.949302,1.151964
commodities:HG=F,-1.132157,-1.153881,-1.126235,1.077268,0.697207,-0.957182,-1.154061,1.043966,-0.644790
commodities:SI=F,0.369445,0.539275,0.342448,-0.178609,0.448531,1.037922,0.543756,-0.094664,-0.507174


In [438]:
rolling[0]['y_ret'].head(3)

commodities:GC=F   -0.001869
commodities:HG=F    0.001322
commodities:SI=F   -0.017257
dtype: float64

In [439]:
def panel_from_windows(windows, x_key="past_weekly_returns", y_key="y_ret"):
    X_list, y_list = [], []
    meta_rows = []

    for w_idx, w in enumerate(windows):
        if x_key == 'past_weekly_returns':
            X_df = w[x_key].T          # assets x n_lookback (weeks) (DataFrame)
        elif x_key == 'X_feat':
            X_df = w[x_key]          # assets x n_features (DataFrame)
        y_ser = w[y_key]           # assets (Series or array)

        if not isinstance(y_ser, pd.Series):
            y_ser = pd.Series(y_ser, index=X_df.index)

        assets = X_df.index.intersection(y_ser.index)
        Xw = X_df.loc[assets].to_numpy(dtype=np.float32)
        yw = y_ser.loc[assets].to_numpy(dtype=np.float32)

        mask = np.isfinite(Xw).all(axis=1) & np.isfinite(yw)
        Xw, yw = Xw[mask], yw[mask]
        assets_kept = assets.to_numpy()[mask]

        X_list.append(Xw)
        y_list.append(yw)

        t0, t1 = w.get("t0", None), w.get("t1", None)
        for a in assets_kept:
            meta_rows.append((w_idx, a, t0, t1))

    X = np.vstack(X_list) ## weeks*assets x n_lookback/n_features
    y = np.concatenate(y_list)
    meta = pd.DataFrame(meta_rows, columns=["window_idx", "asset", "t0", "t1"])
    return X, y, meta

In [ ]:
X_past_returns, y, meta = panel_from_windows(rolling, x_key='past_weekly_returns')
X, y, meta = panel_from_windows(rolling, x_key='past_weekly_returns')
# X, y, meta = panel_from_windows(rolling, x_key='X_feat')
X.shape, y.shape, meta.head() ## len(rolling)*n_asset -- each row is a feature set -- to predict y

((1296, 9),
 (1296,),
    window_idx             asset         t0         t1
 0           0  commodities:GC=F 2022-06-24 2022-06-28
 1           0  commodities:HG=F 2022-06-24 2022-06-28
 2           0  commodities:SI=F 2022-06-24 2022-06-28
 3           1  commodities:GC=F 2022-06-28 2022-06-30
 4           1  commodities:HG=F 2022-06-28 2022-06-30)

In [441]:
windows = rolling  # your rolling list

W = meta["window_idx"].nunique()
split_w = int(0.8 * W)

train_mask = (meta["window_idx"] < split_w).values
test_mask  = (meta["window_idx"] >= split_w).values

X_train_raw, y_train = X[train_mask], y[train_mask]
X_test_raw,  y_test  = X[test_mask],  y[test_mask]
X_past_returns_test_raw = X_past_returns[test_mask]

# small validation from the tail of the training windows
val_w = max(int(0.1 * split_w), 1)
val_start = split_w - val_w
val_mask = ((meta["window_idx"] >= val_start) & (meta["window_idx"] < split_w)).values
tr2_mask = (meta["window_idx"] < val_start).values

X_tr_raw, y_tr = X[tr2_mask], y[tr2_mask]
X_va_raw, y_va = X[val_mask], y[val_mask]

scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr_raw).astype(np.float32)
X_va = scaler.transform(X_va_raw).astype(np.float32)
X_te = scaler.transform(X_test_raw).astype(np.float32)

y_tr = y_tr.astype(np.float32)
y_va = y_va.astype(np.float32)
y_te = y_test.astype(np.float32)

## SCALE Y
y_mean = y_tr.mean()
y_std  = y_tr.std() + 1e-8

y_tr_s = ((y_tr - y_mean) / y_std).astype(np.float32)
y_va_s = ((y_va - y_mean) / y_std).astype(np.float32)
float(y_std)

# X_tr = np.tanh(X_tr)
# X_va = np.tanh(X_va)
# X_te = np.tanh(X_te)

0.014712454751133919

In [442]:
X_tr.shape, X_va.shape, X_te.shape, y_tr.shape, y_va.shape, y_te.shape, f"{y_te.shape[0]/close_df.shape[1]} test periods"

((933, 9), (102, 9), (261, 9), (933,), (102,), (261,), '87.0 test periods')

In [443]:
def build_mlp(in_dim, hidden=(64, 32), dropout=0.1, lr=1e-3):
    inputs = keras.Input(shape=(in_dim,))
    x = inputs
    for h in hidden:
        x = layers.Dense(h, activation="relu",)(x) #kernel_regularizer=regularizers.l2(1e-4)
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=[keras.metrics.MeanAbsoluteError(name="mae")]
    )
    return model

model = build_mlp(in_dim=X_tr.shape[1], hidden=(32, 16), dropout=0.1, lr=1e-4)
# model = build_mlp(in_dim=X_tr.shape[1], hidden=(15, 15, 15, 15, 15, 15), dropout=0.0, lr=1e-4)
# model = build_mlp(in_dim=X_tr.shape[1], hidden=(15, 15, 15, 15, 15, 15), dropout=0.0, lr=0.01)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=100, restore_best_weights=True
    )
]

history = model.fit(
    # X_tr, y_tr,
    X_tr, y_tr_s,   ## Scaled y
    # validation_data=(X_va, y_va),
    validation_data=(X_va, y_va_s),   ## Scaled y
    # epochs=150,
    epochs=100,
    # batch_size=256,
    batch_size=100,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.3046 - mae: 0.8552 - val_loss: 1.1037 - val_mae: 0.8108
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2771 - mae: 0.8362 - val_loss: 1.0728 - val_mae: 0.8005
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.2756 - mae: 0.8375 - val_loss: 1.0436 - val_mae: 0.7906
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2325 - mae: 0.8227 - val_loss: 1.0169 - val_mae: 0.7813
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2081 - mae: 0.8143 - val_loss: 0.9922 - val_mae: 0.7728
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1897 - mae: 0.8075 - val_loss: 0.9698 - val_mae: 0.7648
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1768 - mae: 0.8007 - val_loss: 0.9486 - val_mae: 0.7569
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1549 - mae: 0.7996 - val_loss: 0.9292 - val_mae: 0.7495
Epoch 9/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1

In [444]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

def prediction_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    sign_acc = (np.sign(y_true) == np.sign(y_pred)).mean()
    corr = np.corrcoef(y_true, y_pred)[0, 1] if len(y_true) > 1 else np.nan
    
    return {"MSE": round(mse, 5), "MAE": round(mae, 5), "R2": round(r2, 5), "SignAcc": round(float(sign_acc), 5), "Corr": round(float(corr), 5)}


In [445]:
y_pred_sm = X_past_returns_test_raw.astype(np.float32).mean(axis=1)
# X_te_raw = scaler.inverse_transform(X_te)
# y_pred_sm = X_te_raw.mean(axis=1)
print("Sample Mean")
print(prediction_metrics(y_te, y_pred_sm))

# y_pred = model.predict(X_te, batch_size=1024).squeeze()
## Scaled y:
pred_s = model.predict(X_te, batch_size=1024).squeeze()
y_pred = y_mean + y_std * pred_s

print("MLP")
print(prediction_metrics(y_te, y_pred))

print(y_te.mean())

Sample Mean
{'MSE': 0.00052, 'MAE': 0.01233, 'R2': -0.01059, 'SignAcc': 0.54406, 'Corr': 0.00561}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
MLP
{'MSE': 0.00051, 'MAE': 0.01241, 'R2': -0.00266, 'SignAcc': 0.51724, 'Corr': 0.03108}
0.0011402754


In [431]:
def baseline_sample_mean(test_window):
    # past_weekly_returns: (lookback periods) x (assets)
    return test_window["past_weekly_returns"].mean(axis=0)  # pd.Series indexed by asset

Walk-Forward Evaluation

In [432]:
def walk_forward_eval_mlp(
    rolling,
    train_len=150,          # number of windows to train on each step
    x_key="X_feat",
    y_key="y_ret",
    hidden=(64, 32),
    dropout=0.1,
    lr=3e-4,
    epochs=150,
    batch_size=256,
    seed=42
):
    tf.keras.utils.set_random_seed(seed)

    all_rows = []
    week_metrics = []

    for i in range(train_len, len(rolling)):
        train_windows = rolling[i-train_len:i]
        test_window   = rolling[i]

        # --- build train panel ---
        X_train, y_train, meta = panel_from_windows(train_windows, x_key=x_key, y_key=y_key)
        if X_train.shape[0] == 0:
            continue

        # --- scaler on TRAIN only ---
        scaler = StandardScaler()
        X_train_sc = scaler.fit_transform(X_train).astype(np.float32)

        # --- build test cross-section ---
        X_test_df = test_window[x_key]            # assets x features
        y_test = test_window[y_key]
        if not isinstance(y_test, pd.Series):
            y_test = pd.Series(y_test, index=X_test_df.index)

        assets = X_test_df.index.intersection(y_test.index).sort_values()
        X_test = X_test_df.loc[assets].to_numpy(np.float32)
        y_true = y_test.loc[assets].to_numpy(np.float32)

        mask = np.isfinite(X_test).all(axis=1) & np.isfinite(y_true)
        assets = assets[mask]
        X_test = X_test[mask]
        y_true = y_true[mask]

        X_test_sc = scaler.transform(X_test).astype(np.float32)

        # --- train model (fresh each step) ---
        model = build_mlp(in_dim=X_train_sc.shape[1], hidden=hidden, dropout=dropout, lr=lr)
        es = keras.callbacks.EarlyStopping(monitor="loss", patience=15, restore_best_weights=True)

        model.fit(
            X_train_sc, y_train,
            epochs=epochs,
            batch_size=batch_size,
            verbose=0,
            callbacks=[es]
        )

        # --- predict ---
        y_pred = model.predict(X_test_sc, batch_size=1024, verbose=0).squeeze()

        # --- baseline: sample mean of past period returns ---
        y_pred_sm_ser = baseline_sample_mean(test_window).loc[assets]
        y_pred_sm = y_pred_sm_ser.to_numpy(np.float32)

        # store per-asset predictions
        for a, yt, yp, ypsm in zip(assets, y_true, y_pred, y_pred_sm):
            all_rows.append(
                {"window_idx": i, "asset": a, "y": float(yt), "pred_mlp": float(yp), "pred_sm": float(ypsm)}
            )

        # per-window metrics (cross-section)
        m_mlp = prediction_metrics(y_true, y_pred)
        m_sm  = prediction_metrics(y_true, y_pred_sm)
        week_metrics.append({"window_idx": i, **{f"mlp_{k}": v for k,v in m_mlp.items()},
                                **{f"sm_{k}": v for k,v in m_sm.items()}})

    preds_df = pd.DataFrame(all_rows)
    week_df  = pd.DataFrame(week_metrics)

    # pooled metrics over all (window, asset) test points
    pooled_mlp = prediction_metrics(preds_df["y"], preds_df["pred_mlp"])
    pooled_sm  = prediction_metrics(preds_df["y"], preds_df["pred_sm"])

    return preds_df, week_df, pooled_mlp, pooled_sm

In [433]:
# preds_df, week_df, pooled_mlp, pooled_sm = walk_forward_eval_mlp(
#     rolling=rolling,
#     train_len=100,   # e.g., last 52 periods (with days_per_week=2 that's ~104 trading days)
#     x_key="X_feat",
#     y_key="y_ret",
#     hidden=(16,8)
# )

# print("Pooled MLP:", pooled_mlp)
# print("Pooled  SM:", pooled_sm)
# print("Weekly win-rate (MAE):", (week_df["mlp_MAE"] < week_df["sm_MAE"]).mean())